In [2]:
import pandas as pd
import numpy as np
from gurobipy import *
from itertools import combinations
import time

In [2]:
# cargar la cinética de recuperacion
data_name = 'marvin'
cinetica_path = '../Daniel_inputs/'
block_path = '../minelib_inputs/' + data_name + '.blocks'
prec_path = '../minelib_inputs/'+ data_name +'.prec'
cinetica_file = 'REC_MODEL_T240.csv'
cinetica_df = pd.read_csv(cinetica_path + cinetica_file)
blocks_df = pd.read_csv(block_path, header=None)
rec = cinetica_df['rec90'] # panda series para la recuperacion en t=90

In [4]:
cinetica_df

,x,y,z,rec1,rec2,rec3,rec4,rec5,rec6,rec7,...,rec81,rec82,rec83,rec84,rec85,rec86,rec87,rec88,rec89,rec90
0,1,1,1,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
1,1,1,2,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
2,1,1,3,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
3,1,1,4,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
4,1,1,5,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
5,1,1,6,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
6,1,1,7,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
7,1,1,8,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
8,1,1,9,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693
9,1,1,10,6.596,7.925,10.063,15.273,28.334,39.533,50.285,...,85.065,85.632,86.029,86.316,86.594,86.767,86.890,86.942,86.868,86.693


In [24]:
# histograma de la recuperacion en el instante 90
hist = cinetica_df.hist(column='rec90', bins=5)
# calcular nuevos valores de la funcion objetivo
# usando la recuperación al dia 90
nblocks = blocks_df.shape[0]
# algunos datos dados
c_conv = 2204.62
rec_AU = 0.88
price_AU = 12
seilling_AU = 0.2
rec_CU = 0.77
price_CU = 20
seilling_CU = 7.2
proc_cost = 4
mine_cost = 0.9 # por tonn
bv_list_cinetic = list()
bv_list_calc = list()
economic_value_list = list()
economic_value_rec_list = list()
with open(block_path, 'r') as bm:
    for i in range(nblocks):
        #recuperar datos de bm para calcular valor de cada bloque
        linea_bm = bm.readline()
        linea_lista = linea_bm.split()
        tonn = float(linea_lista[4])
        AU = float(linea_lista[5])
        CU = float(linea_lista[6])
        economic_value = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec[i]/100*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_rec = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec_CU*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_list.append(economic_value)
        economic_value_rec_list.append(economic_value_rec)

In [25]:
# optimo con recuperacion del ultimo dia 
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()
blocks_id_rec90 = [i for i in range(nblocks) if x[i].x==1] # recuperar upit

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 3052245.7535
Found heuristic solution: objective 2.155104e+07
Found heuristic solution: objective 3.447631e+07
Presolve removed 570931 rows and 45413 columns (presolve time = 5s) ...
Presolve removed 617931 rows and 49095 columns
Presolve time: 7.48s
Presolved: 32700 rows, 4176 columns, 65400 nonzeros
Found heuristic solution: objective 8.477468e+08
Variable types: 0 continuous, 4176 integer (4176 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5500000e+09   0.000000e+00   4.745000e+03      8s
    5515    1.2375244e+09   0.000000e+00   0.000000e+00      8s
    5515    1.2375244e+09   0.000000e+00   0.000000e

In [5]:
# optimo con recuperacion cte
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_rec_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2796059.4414
Found heuristic solution: objective 2.031152e+07
Found heuristic solution: objective 3.235205e+07
Presolve removed 576769 rows and 45817 columns (presolve time = 5s) ...
Presolve removed 614385 rows and 48755 columns
Presolve time: 6.96s
Presolved: 36246 rows, 4516 columns, 72492 nonzeros
Found heuristic solution: objective 7.999334e+08
Variable types: 0 continuous, 4516 integer (4516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5643467e+09   0.000000e+00   5.157000e+03      7s
    5659    1.2406584e+09   0.000000e+00   0.000000e+00      7s
    5659    1.2406584e+09   0.000000e+00   0.000000e

In [3]:
# obtener funcion objetivo del .pcpsp, para comparar
# el resultado del modelo gurobi con el modelo AMPL
data_name = 'newman1'
pcpsp_path = '../minelib_inputs/' + data_name + '.pcpsp'
prec_path = '../minelib_inputs/' + data_name + '.prec'
upit_path = '../minelib_inputs/'+ data_name + '.upit'
objective_function_pcpsp = {}
resource_constraint_ub_limits = {}
resource_constraint_lb_limits = {}
resource_constraint_coefficients = {}
with open(pcpsp_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        if linea_lista[0] == 'NAME:':
            dato = linea_lista[1].strip('\n')
            name = dato
        elif linea_lista[0] == 'NBLOCKS:':
            dato = linea_lista[1].strip('\n')
            nblocks = int(dato)
        elif linea_lista[0] == 'NPERIODS:':
            dato = linea_lista[1].strip('\n')
            nperiods = int(dato)
        elif linea_lista[0] == 'NDESTINATIONS:':
            dato = linea_lista[1].strip('\n')
            ndestinations = int(dato)
        elif linea_lista[0] == 'NRESOURCE_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            nresource_side_constraints = int(dato)
        elif linea_lista[0] == 'NGENERAL_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            ngeneral_side_constraints = int(dato)
        elif linea_lista[0] == 'DISCOUNT_RATE:':
            dato = linea_lista[1].strip('\n')
            discount_rate = float(dato)
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_LIMITS:':
            for r in range(nresource_side_constraints):
                for t in range(nperiods):
                    linea = f.readline()
                    lista = linea.split()
                    resource_constraint_ub_limits[r,t] = int(lista[-1])
                    resource_constraint_lb_limits[r,t] = '-Infinity' # falta hacer para el caso general
    
        elif linea_lista[0] == 'OBJECTIVE_FUNCTION:':
            for b in range(nblocks):
                linea = f.readline()
                lista= linea.split()
                objective_function_pcpsp[b,0] = float(lista[1])
                objective_function_pcpsp[b,1] = float(lista[2])
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_COEFFICIENTS:':
                for linea in f:
                    if linea == 'EOF\n':
                        break
                    lista = linea.split()
                    b = int(lista[0])
                    d = int(lista[1])
                    r = int(lista[2])
                    resource_constraint_coefficients[b,r,d] = float(lista[3])

# llenar con ceros las entradas de resource_constraint_coefficients
# que no está definidas
for b,r,d in itertools.product(range(nblocks), range(nresource_side_constraints),range(ndestinations)):
    if not (b,r,d) in resource_constraint_coefficients:
        resource_constraint_coefficients[b,r,d] = 0

print('Data base name: %s' % (name))
print('NBLOCKS: %d' % nblocks)
print('NPERIODS: %d' % nperiods)
print('NDESTINATIONS: %d' % ndestinations)
print('NRESOURCE_SIDE_CONSTRAINTS: %d' % nresource_side_constraints)
print('NGENERAL_SIDE_CONSTRAINTS: %d' % ngeneral_side_constraints)
print('DISCOUNT_RATE: %.2f' % discount_rate)

Data base name: Newman1
NBLOCKS: 1060
NPERIODS: 6
NDESTINATIONS: 2
NRESOURCE_SIDE_CONSTRAINTS: 2
NGENERAL_SIDE_CONSTRAINTS: 0
DISCOUNT_RATE: 0.08


In [4]:
bv_list = list() # block value list para upit
with open(upit_path, 'r') as f:
    for i in range(4):
        f.readline()
    for line in f:
        if not line == 'EOF\n':
            lista = line.split()
            bv_list.append(float(lista[1]))

In [5]:
# beneficio con valores objetivos dado por .upit
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(LinExpr([bv_list[i]for i in range(nblocks)], [x[i] for i in range(nblocks)]), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()
# recuperar upit
blocks_id_upit = [i for i in range(nblocks) if x[i].x==1] # recuperar upit
print('UPIT BLOCKS: %d' % len(blocks_id_upit))

Academic license - for non-commercial use only
Optimize a model with 3922 rows, 1060 columns and 7844 nonzeros
Variable types: 0 continuous, 1060 integer (1060 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 989307.55494
Found heuristic solution: objective 2.608690e+07
Presolve removed 3922 rows and 1060 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 2.60869e+07 989308 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.608689902597e+07, best bound 2.608689902597e+07, gap 0.0000%
UPIT BLOCKS: 1059


In [ ]:
blocks = blocks_id_upit
blockTimesDest = list(itertools.product(blocks,range(ndestinations)))
resourceTimesPeriod = list(itertools.product(list(range(nresource_side_constraints)),
                                             list(range(nperiods))))
blocks_prime = list(itertools.product(blocks, list(range(ndestinations)),list(range(nperiods))))
# resolver pcpsp directamente
x = {}
y = {}
modelo_pcpsp = Model()
for b in blocks:
    for t in range(nperiods):
        x[b,t] = modelo_pcpsp.addVar(vtype=GRB.BINARY, name = "x[%d,%d]" % (b,t))
for b,d,t in blocks_prime:
    y[b,d,t] = modelo_pcpsp.addVar(vtype=GRB.CONTINUOUS, name = "y[%d,%d,%d]" % (b,d,t))

modelo_pcpsp.update()
obj_fn = quicksum([(1/(1+discount_rate))**t*objective_function_pcpsp[b,d]*y[b,d,t] for b,d,t in blocks_prime])
modelo_pcpsp.setObjective(obj_fn)

In [101]:
# funcion de reformulacion
# para PCP_at a PCP_by
def at_by_key(b,d,t):
    if d == 0 and t > 0:
        return b,ndestinations-1,t-1
    elif d > 0:
        return b,d-1,t
    elif d == 0 and t == 0:
        print('at_by_key no esta difinida para los valores d = %d, t = %d' % (d,t))

In [72]:
# ALgoritmo BZ
# valores iniciales
blocks = blocks_id_upit
mu = {}
C = {}
mu[0] = {}
blockTimesDest = list(itertools.product(blocks,range(ndestinations)))
resourceTimesPeriod = list(itertools.product(list(range(nresource_side_constraints)),
                                             list(range(nperiods))))
blocks_prime = list(itertools.product(blocks, list(range(ndestinations)),list(range(nperiods))))
for r,t in resourceTimesPeriod:
    mu[0][r,t] = 0
C[1,0] = set(blocks_prime)
k = 1
l = 1
# calculamos la forma equivalente del problema
# PCP_at, llamada PCP_by.
# funcion objetivo
c = {}

for (b,d,t) in blocks_prime:
    c[b,d,t] = (1.0/(1.0 + discount_rate))**t * objective_function_pcpsp[b,d]

# recalculamos c para PCP_by, c_hat
c_hat = {}
for (b,d,t) in blocks_prime:
    if d > 0 or t >0:
        c_hat[b,d,t] = c[b,d,t]
        c_hat[at_by_key(b,d,t)] = -c[b,d,t]
    else:
        c_hat[b,d,t] = c[b,d,t]
# modelo L(PCP_by,mu[k-1])
m = Model()
# definir variable de PCP_by
for (b,d,t) in blocks_prime:
    x[b,d,t] = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name = "x(%d,%d,%d)" % (b,d,t))
m.update()
# agregar restricciones de
# precedencias.
for t in range(nperiods-1):
    for b in blocks:
        m.addConstr(x[b,ndestinations-1,t] <= x[b,0,t+1])

for d in range(ndestinations-1):
    for t in range(nperiods):
        for b in blocks:
            m.addConstr(x[b,d,t] <= x[b,d+1,t])

with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        a = int(linea_lista[0])
        if a in blocks:
            for j in range(nvecinos):
                b = int(linea_lista[j+2])
                for t in range(nperiods):
                    m.addConstr(x[a,ndestinations-1,t] <= x[b,ndestinations-1,t])

# Funcion objetivo de L(PCP_by,mu[k-1])
cx = LinExpr([c_hat[b,d,t] for b,d,t in blocks_prime],[x[b,d,t] for b,d,t in blocks_prime])
suma = {}
q = resource_constraint_coefficients
d_rhs = resource_constraint_ub_limits 
expr = LinExpr()
for r in range(nresource_side_constraints):
    for t in range(1,nperiods):
        suma[r,t] = mu[k-1][r,t]*(quicksum([q[b,r,d]*(x[b,d,t]-x[at_by_key(b,d,t)]) for b,d in blockTimesDest])-d_rhs[r,t])
        expr += suma[r,t]
    suma[r,0] = mu[k-1][r,0]*(quicksum([q[b,r,d]*(x[b,d,0]-x[at_by_key(b,d,0)]) for b,d in blockTimesDest if d>0])+quicksum([q[b,r,0]*x[b,0,0]for b in blocks])-d_rhs[r,0])
    expr += suma[r,0]

fn_objetivo = cx + expr
m.setObjective(fn_objetivo, GRB.MAXIMIZE)
m.optimize()
# obtenemos w[k]
w = {}
w[k] = x
# STEP 2: chequear optimalidad
if k >= 2:
    for h in range(1,l+1):
        for (b1,d1,t1),(b2,d2,t2) in combinations(C[h,k-1],2):
            if not x[b1,d1,t1] == x[b2,d2,t2]:
                print('no se cumple la restriccion')
        print('True')
    print(time.time()-t_init)
#STEP 3: Encontrar particion de blocks_prime
I = [(b,d,t) for (b,d,t) in blocks_prime if w[k][b,d,t].x == 1]
O = [(b,d,t) for (b,d,t) in blocks_prime if w[k][b,d,t].x == 0]
count = 1
for h in range(1,l+1):
    d = 0 
    if C[h,k-1].intersection(I):
        C[count,k] = C[h,k-1].intersection(I)
        d = 1
    if C[h,k-1].intersection(O):
        if d == 0:
            C[count,k] = C[h,k-1].intersection(O)
            count += 1
        else:
            C[count+1,k] = C[h,k-1].intersection(O)
            count += 2
l = count-1
print('Actualizacion de l <- %d' % l)
# STEP 4: resolver P2k

Optimize a model with 2050204 rows, 340640 columns and 4100408 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 80048 rows and 161266 columns (presolve time = 5s) ...
Presolve removed 80085 rows and 162635 columns
Presolve time: 6.63s
Presolved: 260555 rows, 2228209 columns, 4195863 nonzeros

Ordering time: 0.00s

Barrier performed 0 iterations in 84.94 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 409655 iterations and 85.02 seconds
Optimal objective  1.622863614e+09
Actualizacion de l <- 2


In [79]:
# resolver P2k
model_p2k = Model()
lmbda = {}
for i in range(1,l+1):
    lmbda[i] = model_p2k.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name = "lambda%d" % i)

model_p2k.update()
c_lmbda = {}
init_time = time.time()
for j in range(1,l+1):
    c_lmbda[j] = quicksum([c_hat[b,d,t] for (b,d,t) in blocks_prime if (b,d,t) in C[j,k]])
print('Calculo c_lmbda: %f' % (time.time()-init_time))
obj_p2k = quicksum([c_lmbda[j]*lmbda[j] for j in range(1,l+1)])
model_p2k.setObjective(obj_p2k)

# agregamos las restricciones
# para P2k con el cambio de variable
# agregar restricciones de
# precedencias.
for t in range(nperiods-1):
    for b in blocks:
        model_p2k.addConstr(quicksum([lmbda[j]*((b,ndestinations-1,t) in C[j,k]) for j in range(1,l+1)]) <= 
                            quicksum([lmbda[j]*((b,0,t+1) in C[j,k]) for j in range(1,l+1)]))
for d in range(ndestinations-1):
    for t in range(nperiods):
        for b in blocks:
            model_p2k.addConstr(quicksum([lmbda[j]*((b,d,t) in C[j,k]) for j in range(1,l+1)]) <= 
                                 quicksum([lmbda[j]*((b,d+1,t) in C[j,k]) for j in range(1,l+1)]))

with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        a = int(linea_lista[0])
        if a in blocks:
            for j in range(nvecinos):
                b = int(linea_lista[j+2])
                for t in range(nperiods):
                    model_p2k.addConstr(quicksum([lmbda[j]*((a,ndestinations-1,t) in C[j,k]) for j in range(1,l+1)]) <= 
                                quicksum([lmbda[j]*((b,ndestinations-1,t) in C[j,k]) for j in range(1,l+1)]))

Calculo c_lmbda: 0.404891


In [84]:
# agregar side constraints
Dx_p2k = {}
for r in range(nresource_side_constraints):
    for t in range(1,nperiods):
        Dx_p2k[r,t] = quicksum([q[b,r,d]*(quicksum([lmbda[j]*((b,d,t) in C[j,k]) for j in range(1,l+1)])-
                                          quicksum([lmbda[j]*(at_by_key(b,d,t) in C[j,k]) for j in range(1,l+1)])) for b,d in blockTimesDest]) - d_rhs[r,t]
    Dx_p2k[r,0] = quicksum([q[b,r,d]*(quicksum([lmbda[j]*((b,d,0) in C[j,k]) for j in range(1,l+1)])-
                                      quicksum([lmbda[j]*(at_by_key(b,d,0) in C[j,k]) for j in range(1,l+1)])) for b,d in blockTimesDest if d>0]) + quicksum([q[b,r,0]*quicksum([lmbda[j]*((b,0,0) in C[j,k]) for j in range(1,l+1)]) for b in blocks]) - d_rhs[r,0]

# agregar side constraints Dx <= d
side_const = {}
for r in range(nresource_side_constraints):
    for t in range(nperiods):
        side_const[r,t] = model_p2k.addConstr(Dx_p2k[r,t] <= 0, name='side_const[%d,%d]' % (r,t))

In [85]:
model_p2k.optimize()

Optimize a model with 2050324 rows, 2 columns and 8006 nonzeros
Coefficient statistics:
  Matrix range     [8e-07, 5e+08]
  Objective range  [2e+09, 2e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 6e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.43 seconds
Optimal objective  0.000000000e+00


In [138]:
# BZ (notar que labda fue reescalado y los
# coeficientes de la función objetivo tambien)
blocks = blocks_id_upit
mu = {}
C = {}
mu[0] = {}
blockTimesDest = list(itertools.product(blocks,range(ndestinations)))
resourceTimesPeriod = list(itertools.product(list(range(nresource_side_constraints)),
                                             list(range(nperiods))))
blocks_prime = list(itertools.product(blocks, list(range(ndestinations)),list(range(nperiods))))
for r,t in resourceTimesPeriod:
    mu[0][r,t] = 0
C[1,0] = set(blocks_prime)
k = 1
l = 1
# calculamos la forma equivalente del problema
# PCP_at, llamada PCP_by.
# funcion objetivo
c = {}
for (b,d,t) in blocks_prime:
    c[b,d,t] = (1.0/(1.0 + discount_rate))**t * objective_function_pcpsp[b,d]

# recalculamos c para PCP_by, c_hat
c_hat = {}
for (b,d,t) in blocks_prime:
    if d > 0 or t >0:
        c_hat[b,d,t] = c[b,d,t]
        c_hat[at_by_key(b,d,t)] = -c[b,d,t]
    else:
        c_hat[b,d,t] = c[b,d,t]

# modelo L(PCP_by,mu[k-1])
maestro = Model()
# definir variable de PCP_by
for (b,d,t) in blocks_prime:
    x[b,d,t] = maestro.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name = "x(%d,%d,%d)" % (b,d,t))
maestro.update()
# agregar restricciones de
# precedencias.
for t in range(nperiods-1):
    for b in blocks:
        maestro.addConstr(x[b,ndestinations-1,t] <= x[b,0,t+1])

for d in range(ndestinations-1):
    for t in range(nperiods):
        for b in blocks:
            maestro.addConstr(x[b,d,t] <= x[b,d+1,t])

with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        a = int(linea_lista[0])
        if a in blocks:
            for j in range(nvecinos):
                b = int(linea_lista[j+2])
                for t in range(nperiods):
                    maestro.addConstr(x[a,ndestinations-1,t] <= x[b,ndestinations-1,t])

# Funcion objetivo de L(PCP_by,mu[k-1])
cx = LinExpr([c_hat[b,d,t] for b,d,t in blocks_prime],[x[b,d,t] for b,d,t in blocks_prime])
q = resource_constraint_coefficients
d_rhs = resource_constraint_ub_limits
w = {}
z = {}
while True:
    # STEP 1: resolver L(PCPby,mu[k-1])
    suma = {}
    expr = LinExpr()
    for r in range(nresource_side_constraints):
        for t in range(1,nperiods):
            suma[r,t] = mu[k-1][r,t]*(quicksum([q[b,r,d]*(x[b,d,t]-x[at_by_key(b,d,t)]) for b,d in blockTimesDest])-d_rhs[r,t])
            expr += suma[r,t]
        suma[r,0] = mu[k-1][r,0]*(quicksum([q[b,r,d]*(x[b,d,0]-x[at_by_key(b,d,0)]) for b,d in blockTimesDest if d>0])+quicksum([q[b,r,0]*x[b,0,0]for b in blocks])-d_rhs[r,0])
        expr += suma[r,0]

    fn_objetivo = cx + expr
    maestro.setObjective(fn_objetivo, GRB.MAXIMIZE)
    maestro.Params.Presolve = 0
    maestro.optimize()
    w[k] = x
    if k >= 2:
        parar = 0
        for h in range(1,l+1):
            for (b1,d1,t1),(b2,d2,t2) in combinations(C[h,k-1],2):
                if not w[k][b1,d1,t1] == w[k][b2,d2,t2]:
                    parar = 1
        if parar == 0:
            print('Algoritmo termino: H^%d w[%d] = 0' % (k-1,k))
            break
    #STEP 3: encontrar particion de blocks_prime
    I = [(b,d,t) for (b,d,t) in blocks_prime if w[k][b,d,t].x == 1]
    O = [(b,d,t) for (b,d,t) in blocks_prime if w[k][b,d,t].x == 0]
    count = 1
    for h in range(1,l+1):
        d = 0 
        if C[h,k-1].intersection(I):
            C[count,k] = C[h,k-1].intersection(I)
            d = 1
        if C[h,k-1].intersection(O):
            if d == 0:
                C[count,k] = C[h,k-1].intersection(O)
                count += 1
            else:
                C[count+1,k] = C[h,k-1].intersection(O)
                count += 2
    l = count-1
    # STEP 4: resolver P2k
    model_p2k = Model()
    lmbda = {}
    var_scale = 1
    obj_scale = 1
    for i in range(1,l+1):
        lmbda[i] = model_p2k.addVar(lb=0, ub=var_scale, vtype=GRB.CONTINUOUS, name = "lambda%d" % i)

    model_p2k.update()
    c_lmbda = {}
    for j in range(1,l+1):
        c_lmbda[j] = quicksum([c_hat[b,d,t] for (b,d,t) in blocks_prime if (b,d,t) in C[j,k]])
    obj_p2k = quicksum([obj_scale*c_lmbda[j]*lmbda[j] for j in range(1,l+1)])
    model_p2k.setObjective(obj_p2k)

    # agregamos las restricciones
    # para P2k con el cambio de variable
    # agregar restricciones de
    # precedencias.
    for t in range(nperiods-1):
        for b in blocks:
            model_p2k.addConstr(quicksum([lmbda[j]*((b,ndestinations-1,t) in C[j,k]) for j in range(1,l+1)]) <= 
                                quicksum([lmbda[j]*((b,0,t+1) in C[j,k]) for j in range(1,l+1)]))
    for d in range(ndestinations-1):
        for t in range(nperiods):
            for b in blocks:
                model_p2k.addConstr(quicksum([lmbda[j]*((b,d,t) in C[j,k]) for j in range(1,l+1)]) <= 
                                     quicksum([lmbda[j]*((b,d+1,t) in C[j,k]) for j in range(1,l+1)]))

    with open(prec_path, 'r') as f:
        for linea in f:
            linea_lista = linea.split()
            nvecinos = int(linea_lista[1])
            a = int(linea_lista[0])
            if a in blocks:
                for j in range(nvecinos):
                    b = int(linea_lista[j+2])
                    for t in range(nperiods):
                        model_p2k.addConstr(quicksum([lmbda[j]*((a,ndestinations-1,t) in C[j,k]) for j in range(1,l+1)]) <= 
                                    quicksum([lmbda[j]*((b,ndestinations-1,t) in C[j,k]) for j in range(1,l+1)]))
    
    # agregar side constraints
    Dx_p2k = {}
    for r in range(nresource_side_constraints):
        for t in range(1,nperiods):
            Dx_p2k[r,t] = quicksum([q[b,r,d]*(quicksum([lmbda[j]*((b,d,t) in C[j,k]) for j in range(1,l+1)])-
                                              quicksum([lmbda[j]*(at_by_key(b,d,t) in C[j,k]) for j in range(1,l+1)])) for b,d in blockTimesDest]) - d_rhs[r,t]
        Dx_p2k[r,0] = quicksum([q[b,r,d]*(quicksum([lmbda[j]*((b,d,0) in C[j,k]) for j in range(1,l+1)])-
                                          quicksum([lmbda[j]*(at_by_key(b,d,0) in C[j,k]) for j in range(1,l+1)])) for b,d in blockTimesDest if d>0]) + quicksum([q[b,r,0]*quicksum([lmbda[j]*((b,0,0) in C[j,k]) for j in range(1,l+1)]) for b in blocks]) - d_rhs[r,0]

    # agregar side constraints Dx <= d
    side_const = {}
    for r in range(nresource_side_constraints):
        for t in range(nperiods):
            side_const[r,t] = model_p2k.addConstr(Dx_p2k[r,t] <= 0, name='side_const[%d,%d]' % (r,t))
    
    model_p2k.Params.Presolve = 0
    model_p2k.optimize()
    # definir z[k]
    mu[k] = {}
    for r in range(nresource_side_constraints):
        for t in range(nperiods):
            mu[k][r,t] = side_const[r,t].pi
    if mu[k] == mu[k-1]: # Cambiar por una tolerancia!!!
        print('Algoritmo termino: mu[%d] = mu[%d]' % (k,k-1))
        break
    k += 1

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Optimize a model with 2050204 rows, 340640 columns and 4100408 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 3.53 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 90380 iterations and 3.59 seconds
Optimal objective  1.622863614e+09
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Optimize a model with 2050244 rows, 2 columns and 7998 nonzeros
Coefficient statistics:
  Matrix range     [8e-07, 5e+08]
  Objective range  [2e+09, 2e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 6e+07]
         Consider reformulating model or setting NumericFocus

In [139]:
model_p2k.Kappa

15980265.437500412

In [130]:
max(zip(c_lmbda.values(), c_lmbda.keys()))

(<gurobi.LinExpr: -1622863614.1527443>, 2)

In [132]:
c_lmbda.values()

dict_values([<gurobi.LinExpr: 1622863614.1527486>, <gurobi.LinExpr: -1622863614.1527443>])

In [123]:
min(zip(c_hat.values(), c_hat.keys()))

(-1291865.69435423, (21052, 0, 0))

In [33]:
quicksum([c_hat[b,d,t] for (b,d,t) in blocks_prime])

<gurobi.LinExpr: 1.856096787378192e-08>

In [58]:
xx = {}
largo = 1000
blocks_set = set(blocks_prime[:largo])
C_set = set(C[1,1])

In [59]:
for (b,d,t) in blocks_set:
    if (b,d,t) in C_set:
        xx[b,d,t] = 1
    else:
        xx[b,d,t] = 0

In [ ]:
model_p2k = Model()
x_base = model_p2k.addVars(blocks_prime,obj = )

In [ ]:
# resolver P2k
from collections import defaultdict
model_p2k = Model()
lmbda = {}
for i in range(1,l+1):
    lmbda[i] = model_p2k.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name = "lambda%d" % i)
    
base = {} # base de las restricciones del espacio H^k = 0
for i in range(1,l+1):
    base[i]= {}
    for (b,d,t) in blocks_prime:
        if (b,d,t) not in C[i,k]:
            base[i][b,d,t] = 0
        else:
            base[i][b,d,t] = 1
c_lmbda = {}
for j in range(1,l+1):
    c_lmbda = sum([c_hat[b,d,t]*base[i][b,d,t] for (b,d,t) in blocks_prime])
obj_lmbda = quicksum((*lmbda[i] for i in range(1,l+1))
model_p2k.update()